In [1]:
import seisbench.models as sbm
import obspy
import os
from obspy.core.inventory.inventory import read_inventory
import pandas as pd
from obspy.geodetics.base import gps2dist_azimuth
from obspy.core.utcdatetime import UTCDateTime
import numpy as np

import label_phases as labeler

In [2]:
# Set params
out_base_dir = "." #"/uufs/chpc.utah.edu/common/home/koper-group3/alysha"
experiment_name = "BASE"
source_type = "eq"
stat_list = None # If none, use all stations #"BH2B", "BH2D", "BH3C", 
event_ot_str_list = ["2010-06-21T12:01:03.000000Z"] # If none, use all events
av_p_vel = 6.0 #km/s
av_s_vel = 3.39 #km/s
p_pick_thresh = 0.1
s_pick_thresh = 0.1
save_plots=True
# Number of seconds around the auto group velocity for prioritizing picks and assigning quality
auto_pick_time_delta_s = 15
model = sbm.PhaseNet.from_pretrained("stead")

/uufs/chpc.utah.edu/common/home/u1072028/software/pkg/miniconda3/envs/seisbench/lib/python3.11/site-packages/seisbench/models/base.py:489: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental

In [4]:
np.isin("EQ".lower(), [s.lower() for s in ["ex", "eq"]])

array(True)

In [3]:
main_dir = "/uufs/chpc.utah.edu/common/home/koper-group4/relu/Spectral_modeling/Utah/"
cat_dir_name = "Catalogs_BASE_SSIP_073024"
events_dir_name = "Events_BASE_SSIP_073024"
out_dir_name = "single_fire_picks"

In [4]:
# Complete paths and read in 
col_names = ["phase.AT", "QUAL", "NET.STAT.LOC.CHAN", "N1", "N2", "N3", "PHASE", 
             "N4", "N5", "METHOD", "PEAK_VALUE", "AUTO_DELTA_S"]
cat_file = os.path.join(main_dir, f"{cat_dir_name}/{experiment_name.upper()}/cat_{experiment_name.lower()}_{source_type.lower()}.csv")
events_dir = os.path.join(main_dir, f"{events_dir_name}/Events_{experiment_name.upper()}_{source_type.upper()}")
out_base_dir = os.path.join(out_base_dir, f"{out_dir_name}/{experiment_name.upper()}_{source_type.upper()}")
print("Catalog dir:", cat_file)
print("Events dir:", events_dir)
print("Output dir:", out_base_dir)
cat = pd.read_csv(cat_file)
if event_ot_str_list is not None:
    cat = cat[cat.UTC.isin(event_ot_str_list)]
cat.head()

Catalog dir: /uufs/chpc.utah.edu/common/home/koper-group4/relu/Spectral_modeling/Utah/Catalogs_BASE_SSIP_073024/BASE/cat_base_eq.csv
Events dir: /uufs/chpc.utah.edu/common/home/koper-group4/relu/Spectral_modeling/Utah/Events_BASE_SSIP_073024/Events_BASE_EQ
Output dir: ./single_fire_picks/BASE_EQ


,LAT,LON,DEPTH,MAG,MTYPE,EVENT_TYPE,UTC
0,43.733,-107.101,6.1,1.4,Ml,earthquake,2010-06-21T12:01:03.000000Z


In [5]:
# WARNING: This isn't exactly the same as label_phases.main -> refer to label_phases.main for most up to date code

# If the stat_list is not set, update the list for every event
update_stat_list = False
if stat_list is None:
    update_stat_list = True

# Iterate over the event rows in the catalog
for i, event_row in cat.iterrows():
    # Extract relavent event information
    event_ot_str = event_row["UTC"]
    event_loc = (event_row["LAT"], event_row["LON"])
    event_ot_utc = UTCDateTime(event_ot_str)

    # Set the paths for the waveforms and station information for this event
    wf_dir = os.path.join(events_dir, f"{event_ot_str}/Data/waveforms")
    xml_dir = os.path.join(events_dir, f"{event_ot_str}/Data/stations")
    
    inv = read_inventory(os.path.join(xml_dir, f"*xml"))
    # If the list of station was not specified, use all of them
    if update_stat_list is None:    
        stat_list = np.unique([chan.split(".")[1] for chan in inv.get_contents()["channels"]])

    # Set the and make the output dir for this event. Directory name is the event OT
    out_dir = os.path.join(out_base_dir, event_ot_str)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    # Set the pick output file name as "picks_{experiment}_{source}.txt" (e.g., "picks_base_eq.txt")
    output_file = os.path.join(out_dir, f"picks_{experiment_name.lower()}_{source_type.lower()}.txt")

    # If saving the plots to disk, make directory in event output dir called "plots"
    if save_plots:
        plot_out_dir = os.path.join(out_dir, "plots")
        if not os.path.exists(plot_out_dir):
            os.makedirs(plot_out_dir)

    # Iterate over the stations in the stat_list
    ev_rows = []
    for stat in stat_list:
        # Get the station metadata
        stat_inv = inv.select(station=stat)
        if len(stat_inv) == 0:
            print(f"No {stat} info")
            continue

        # Compute the auto P and S arrival times using the set group velocities
        stat_loc = (stat_inv[0][0].latitude, stat_inv[0][0].longitude)
        sr_dist_km = gps2dist_azimuth(stat_loc[0], stat_loc[1], event_loc[0], event_loc[1])[0]/1000
        p_at = event_ot_utc + sr_dist_km/av_p_vel
        s_at = event_ot_utc + sr_dist_km/av_s_vel

        # Read in the three component waveforms for the station
        st = obspy.read(os.path.join(wf_dir, f"*{stat}*mseed"))
        assert len(st) == 3
        # Get the seisbench model posterior probabilities
        preds = model.annotate(st)
        # Get the picks corresponding to posterior probabilities greater than the pick thresholds
        phase_picks = model.classify(st, **{"P_threshold":p_pick_thresh, "S_threshold":s_pick_thresh})

        # Plot the waveforms and picks
        labeler.plot(st.copy(), p_at, s_at, preds, phase_picks.picks, 
            title=f"{experiment_name.upper()} {source_type.upper()} {event_ot_str} {stat}",
            output_file_name=[os.path.join(plot_out_dir, f"{st[0].id[:-1]}.png") if save_plots else None][0])

        # Get P pick information for the output file 
        p_picks = phase_picks.picks.select(phase="P")
        if len(p_picks) == 0:
             ev_rows.append(labeler.format_auto_df_row(st, p_at, "P"))
        else:
            ev_rows.append(labeler.make_ML_row(st, p_picks, p_at, delta_s=auto_pick_time_delta_s))

        # Get S pick information for the output file
        s_picks = phase_picks.picks.select(phase="S")
        if len(s_picks) == 0:
             ev_rows.append(labeler.format_auto_df_row(st, s_at, "S"))
        else:
            ev_rows.append(labeler.make_ML_row(st, s_picks, s_at, delta_s=auto_pick_time_delta_s))

    df = pd.DataFrame(ev_rows, 
                      columns=col_names)
    labeler.write_output(df, output_file)

DataFrame saved to ./single_fire_picks/BASE_EQ/2010-06-21T12:01:03.000000Z/picks_base_eq.txt


In [6]:
df

,phase.AT,QUAL,NET.STAT.LOC.CHAN,N1,N2,N3,PHASE,N4,N5,METHOD,PEAK_VALUE,AUTO_DELTA_S
0,phase: 2010-06-21 12:01:22.60000,1,XV.BB2..BHZ,None,None,None,P,None,False,ML,0.73,-2.0
1,phase: 2010-06-21 12:01:35.56000,1,XV.BB2..BHZ,None,None,None,S,None,False,ML,0.71,-1.4
2,phase: 2010-06-21 12:01:23.27000,1,XV.BB3..BHZ,None,None,None,P,None,False,ML,0.86,-2.2
3,phase: 2010-06-21 12:01:36.06000,1,XV.BB3..BHZ,None,None,None,S,None,False,ML,0.85,-1.1
4,phase: 2010-06-21 12:01:33.28182,4,XV.BH1A..BHE,None,None,None,P,None,False,GV,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
129,phase: 2010-06-21 12:02:16.70969,4,IU.RSSD.00.BH1,None,None,None,S,None,False,GV,None,0.0
130,phase: 2010-06-21 12:01:40.87487,4,IW.RWWY..BHE,None,None,None,P,None,False,GV,None,0.0
131,phase: 2010-06-21 12:02:10.03518,4,IW.RWWY..BHE,None,None,None,S,None,False,GV,None,0.0
132,phase: 2010-06-21 12:00:43.36000,3,Z2.SNFF..BHZ,None,None,None,P,None,False,ML,0.43,54.0
